In [48]:
import bs4 as bs
import urllib.request
import pandas as pd
import unicodedata
from itertools import cycle 
import re
import time
import numpy as np
from datetime import datetime, timedelta 
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)

sauce = urllib.request.urlopen('https://www.basketball-reference.com/draft/NBA_2018.html').read()

soup = bs.BeautifulSoup(sauce, 'lxml')

# Set file path and select years

In [176]:
path = 'C:/Users/erler/OneDrive/Documents/Random Data Sets/'
#choose years span (though note that it runs extremely slow for many years at a time)
first_year = 2008
last_year = 2011 

# Create table of all NBA picks for a given timespan

In [177]:
ticB1 = time.perf_counter()
years = [i for i in range(first_year, last_year+1)]
p1 = 'https://www.basketball-reference.com/draft/NBA_'
p2 = '.html'
drafts = []
for i in years:
    x = pd.read_html(p1+str(i)+p2)[0].droplevel(0, axis=1) #get each years draft class
    x['Year'] = i #add year to table
    x = x.loc[x.Rk.str.isnumeric()==True] #filter for headers and non-data
    x['pro_link']=np.nan #new
    sauce = urllib.request.urlopen(p1+str(i)+p2).read() #import html data
    soup = bs.BeautifulSoup(sauce, 'lxml') 
    table = soup.table #find table
    table_rows = table.find_all('tr') #find table rows
    for tr in table_rows: #find table links for players page and put in a list
        try:
            pick = int(tr.find_all('td')[0].get('csk')) #get pick number
        except:
            pass
        a = tr.find_all('a')
        for url in a: 
            if url.get('href')[:9] == '/players/':
                x.loc[x['Pk']==str(pick), 'pro_link'] = url.get('href')
                                
    drafts.append(x) #append each year
    
drafts = pd.concat(drafts)  #concat each df/year into one dataframe

#make a player name column that matches url use
drafts.Player = drafts.Player.replace({' ': '-'}, regex=True).str.lower() 
drafts['Player_url'] = [unicodedata.normalize('NFD', t).encode('ascii', 'ignore').decode('utf-8') for t in drafts.Player]
drafts.reset_index(drop=True, inplace=True)

tocB1 = time.perf_counter()
print(f"{tocB1 - ticB1:0.4f} seconds") #revised
#print(f"{tocA1 - ticA1:0.4f} seconds") #original

5.4729 seconds


# Retrieve urls for college or international league gamelogs

In [178]:
ticB2 = time.perf_counter()

def linkfix(x, testlink): #normalize links for website quirks
    if x[-1] != '/':
        x+='/'
    if x[:len(testlink)] == testlink:
        pass
    else:
        x=testlink+x
    return x

drafts['cbb_gamelogs'] = np.nan #set destination column
for j in range(len(drafts)+1): #for all players
    leagues = []
    seasons = []
    try:
        sauce = urllib.request.urlopen('https://www.basketball-reference.com'+drafts.pro_link[j]) #url main page of each player
        soup = bs.BeautifulSoup(sauce, 'lxml') 

        drafts.loc[j, 'Pro_height'] = soup.find_all('span',itemprop="height")[0].text
        drafts.loc[j, 'Pro_weight'] = re.sub("[^0-9]",'', soup.find_all('span',itemprop="weight")[0].text)
        drafts.loc[j, 'DOB'] = [i.get('data-birth') for i in soup.find_all('span') if i.get('data-birth')][0]

        a = soup.find_all('a') #find all links
        testcbb='https://www.basketball-reference.com'  
        testintl='https://www.basketball-reference.com'
        testnba='https://www.basketball-reference.com'
        for url in a:
            try: #retrieve link to gamelogs for college games from pro profile
                if url.get('href')[:45] == 'https://www.sports-reference.com/cbb/players/':
                    linkfull = url.get('href')
                    sauce = urllib.request.urlopen(linkfull) #url main page of each cbb player
                    soup = bs.BeautifulSoup(sauce, 'lxml')
                    drafts.loc[j, 'cbb_height'] = soup.find_all('span',itemprop="height")[0].text #get cbb height
                    drafts.loc[j, 'cbb_weight'] = re.sub("[^0-9]",'', soup.find_all('span',itemprop="weight")[0].text) #get cbb weight
                    link = linkfull[:-5]+'/gamelog/' #adjust urls to get gamelogs
                    leagues.append(linkfix(link, 'https://www.sports-reference.com'))                     

                #retrieve links to gamelogs for international league games from pro profile
                if url.get('href')[:59] == 'https://www.basketball-reference.com/international/players/':
                    link = url.get('href') #get all links to intl play
                    sauce2 = urllib.request.urlopen(link) #get html
                    soup2 = bs.BeautifulSoup(sauce2, 'lxml') 
                    a2 = soup2.find_all('a') #find all links to leagues for each player from intl page
                    for url2 in a2: #get the links for gamelogs
                        try:
                            if 'gamelog' in url2.get('href'):
                                x = url2.get('href')
                                leagues.append(linkfix(x, testintl))

                        except:
                            pass

                #Retrieve NBA gamelogs        
                if 'gamelog' in url.get('href'):
                    x = url.get('href')
                    seasons.append(linkfix(x, 'https://www.basketball-reference.com'))   

            except:
                pass
    except:
        pass    
        
    leagues=set(leagues) #eliminate duplicates
    seasons=set(seasons) #eliminate duplicates
    drafts.loc[j, 'cbb_gamelogs'] = [leagues] #store in player table
    drafts.loc[j, 'pro_gamelogs'] = [seasons] #store in player table
    print(j)
    
tocB2 = time.perf_counter()
print(f"{tocB2 - ticB2:0.4f} seconds")  #revised

#print(f"{tocA2 - ticA2:0.4f} seconds")  #original 

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
989.1148 seconds


In [179]:
#save file just in case the next steps stall out
drafts.to_csv(path+'drafts'+'.csv')

# Assemble pre-NBA gamelogs into database

In [162]:
ticB3 = time.perf_counter()

stats = [] #destination list

for i in range(len(drafts)+1):
    try:
        for q in drafts.loc[i, 'cbb_gamelogs']: #for each league
            x = pd.read_html(q)[0] #pull in gamelogs table
            x['League'] = 'NCAA' if q.split('/')[-2] == 'gamelog' else q.split('/')[-2] #league name
            #conform headers
            x.rename(columns={'School': 'Team', 'Unnamed: 7':'W/L', 'Unnamed: 5':'W/L', 'Unnamed: 4':'Location', 'Unnamed: 3':'Location'}, inplace=True)
            x['Player']=drafts.loc[i,'Player'] #player name
            x['DOB']=drafts.loc[i,'DOB'] #player name
            if x.Rk.dtype != 'O':        #remove junk
                x = x.loc[x.Rk.isna()!=True]
                stats.append(x)
            else:
                x = x.loc[x.Rk.str.isdigit()==True]
                stats.append(x)
    except:
        pass 
    print(i)
Predraft_game_stats = pd.concat(stats) #combine dataframes

tocB3 = time.perf_counter()

print(f"{tocB3 - ticB3:0.4f} seconds")  #revised

#print(f"{tocA3 - ticA3:0.4f} seconds")  #original 

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
639.2855 seconds


In [163]:
Predraft_game_stats.Date = pd.to_datetime(Predraft_game_stats.Date)
Predraft_game_stats.Rk = pd.to_numeric(Predraft_game_stats.Rk)
Predraft_game_stats[Predraft_game_stats.loc[:,'FG':'PTS'].columns.values] = Predraft_game_stats[Predraft_game_stats.loc[:,'FG':'PTS'].columns.values].astype('float64')
Predraft_game_stats.convert_dtypes(infer_objects=False).dtypes

#save file
Predraft_game_stats.to_csv(path+'Predraft_game_stats'+str(first_year)+'_'+str(last_year)+'.csv', index=None)

In [164]:
#Combine previous scrapes and current into one file 
Predraft_game_stats_prior = pd.read_csv(path+'Predraft_game_stats.csv')
Predraft_game_stats_current = pd.read_csv(path+'Predraft_game_stats'+str(first_year)+'_'+str(last_year)+'.csv')
Predraft_game_stats = pd.concat([Predraft_game_stats_prior, Predraft_game_stats_current])
Predraft_game_stats.to_csv(path+'Predraft_game_stats'+'.csv', index=None)

C:\Users\erler\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (34) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Assemble NBA gamelogs into database

In [2]:
#if drafts table needs to be loaded
drafts = pd.read_csv(path+'drafts.csv')
drafts['cbb_gamelogs'] = drafts['cbb_gamelogs'].apply(lambda x: eval(x))
drafts['pro_gamelogs'] = drafts['pro_gamelogs'].apply(lambda x: eval(x))

In [ ]:
ticB3 = time.perf_counter()

NBA_stats = [] #destination list

for i in range(len(drafts)+1):
    try:
        for q in drafts.loc[i, 'pro_gamelogs']: #for each gamelog url
            x = pd.read_html(q)[7] #get table
            #conform column names
            x.rename(columns={'Unnamed: 7':'W/L', 'Unnamed: 8':'W/L', 'Unnamed: 5':'Location', 'Unnamed: 3':'Location'}, inplace=True)
            x.rename(columns=dict(zip(list(x.columns[x.columns.str.contains('Playoffs')==True]), cycle(['Date']))), inplace=True) 
            x['Player']=drafts.loc[i,'Player'] #get player name
            x['DOB']=drafts.loc[i,'DOB'] #get DOB
            if x.Rk.dtype != 'O':  #remove junk rows      
                x = x.loc[x.Rk.isna()!=True]
                NBA_stats.append(x)
            else:
                x = x.loc[x.Rk.str.isdigit()==True]
                NBA_stats.append(x)
    except:
        pass 
    #print(i)
NBA_game_stats = pd.concat(NBA_stats) #combine tables
NBA_game_stats = NBA_game_stats.loc[NBA_game_stats.MP.str.match(r'[^0-9]')!=True]
tocB3 = time.perf_counter()

print(f"{tocB3 - ticB3:0.4f} seconds")  #revised

#print(f"{tocA3 - ticA3:0.4f} seconds")  #original 

In [166]:
NBA_game_stats.to_csv(path+'NBAp'+str(first_year)+'_'+str(last_year)+'.csv', index=None) #save to folder in case failure

In [167]:
NBA_game_stats_prior = pd.read_csv('C:/Users/erler/OneDrive/Documents/Random Data Sets/'+'NBA_game_stats.csv')
NBAp_current = pd.read_csv('C:/Users/erler/OneDrive/Documents/Random Data Sets/'+'NBAp'+str(first_year)+'_'+str(last_year)+'.csv')

C:\Users\erler\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (32,35,37,39,40,41,42,43,44) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [175]:
NBA_game_stats = pd.concat([NBA_game_stats_prior, NBAp_current])
#NBA_game_stats = NBA_game_stats.drop(['Unnamed: 0'], axis=1)
NBA_game_stats.to_csv(path+'NBA_game_stats'+'.csv', index=None)

In [174]:
NBA_game_stats.loc[NBA_game_stats.Player.str.contains('gian')]

,Rk,G,Date,Age,Tm,Location,Opp,W/L,GS,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-,League,Player,2015 Playoffs,Series,G#,2014 Playoffs,Unnamed: 31,2016 Playoffs,Unnamed: 32,2013 Playoffs,2017 Playoffs,2018 Playoffs,2019 Playoffs,2020 Playoffs,DOB
22461,2.0,1.0,2015-10-30,20-328,MIL,NaN,WAS,L (-5),1.0,34:09,9.0,15.0,0.600,1.0,3.0,0.333,8.0,10.0,0.800,3.0,6.0,9.0,2.0,3.0,0.0,6.0,6.0,27.0,19.2,-10.0,NaN,giannis-antetokounmpo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22462,3.0,2.0,2015-11-01,20-330,MIL,@,TOR,L (-19),1.0,32:41,8.0,12.0,0.667,1.0,1.0,1.000,3.0,3.0,1.000,0.0,9.0,9.0,2.0,0.0,1.0,0.0,1.0,20.0,19.2,-7.0,NaN,giannis-antetokounmpo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22463,4.0,3.0,2015-11-02,20-331,MIL,@,BRK,W (+7),1.0,42:12,8.0,16.0,0.500,1.0,3.0,0.333,4.0,5.0,0.800,3.0,5.0,8.0,3.0,0.0,2.0,4.0,4.0,21.0,14.1,2.0,NaN,giannis-antetokounmpo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22464,5.0,4.0,2015-11-04,20-333,MIL,NaN,PHI,W (+4),1.0,40:32,7.0,11.0,0.636,1.0,1.0,1.000,2.0,3.0,0.667,0.0,5.0,5.0,3.0,1.0,1.0,3.0,2.0,17.0,13.2,0.0,NaN,giannis-antetokounmpo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22465,6.0,5.0,2015-11-06,20-335,MIL,@,NYK,W (+7),1.0,35:39,7.0,13.0,0.538,0.0,0.0,NaN,6.0,6.0,1.000,1.0,6.0,7.0,1.0,2.0,1.0,1.0,4.0,20.0,17.0,7.0,NaN,giannis-antetokounmpo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56954,46.0,13.0,2018-01-22,20-203,SAC,@,CHO,L (-5),0.0,12:50,3.0,7.0,0.429,0.0,0.0,NaN,0.0,0.0,NaN,2.0,4.0,6.0,0.0,0.0,0.0,0.0,3.0,6.0,3.7,-8.0,NaN,georgios-papagiannis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56955,47.0,14.0,2018-01-23,20-204,SAC,@,ORL,W (+6),0.0,3:04,0.0,2.0,0.000,0.0,0.0,NaN,0.0,0.0,NaN,1.0,1.0,2.0,0.0,0.0,1.0,1.0,0.0,0.0,-0.7,-9.0,NaN,georgios-papagiannis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56956,50.0,15.0,2018-01-30,20-211,SAC,@,NOP,W (+11),0.0,10:35,1.0,2.0,0.500,0.0,0.0,NaN,0.0,0.0,NaN,0.0,2.0,2.0,4.0,0.0,1.0,1.0,2.0,2.0,3.3,-6.0,NaN,georgios-papagiannis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56957,51.0,16.0,2018-02-02,20-214,SAC,NaN,GSW,L (-15),0.0,1:26,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,georgios-papagiannis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
# NBA_game_stats[NBA_game_stats['PTS'].apply(lambda x: type(x)!=str)].shape
# #pd.to_timedelta(NBA_game_stats['MP'].str.split(':')[0], unit= 'min') 
# NBA_game_stats.PTS.astype('float').sort_values(ascending=False)

46925    70.0
2411     61.0
2433     61.0
5293     60.0
4356     60.0
         ... 
18541     0.0
45574     0.0
18546     0.0
45573     0.0
44952     0.0
Name: PTS, Length: 94923, dtype: float64